In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
%%bash
cd ..
pwd
cd data
ls
pwd

/home/naqib/Documents/git/data-engineering-projects/nosql-tutorial
201901-citibike-tripdata.csv
201901-citibike-tripdata.csv.zip
201902-citibike-tripdata.csv.zip
201903-citibike-tripdata.csv.zip
city_inspections.json
data2.json
data.json
gls.json
zips.json
/home/naqib/Documents/git/data-engineering-projects/nosql-tutorial/data


In [37]:
df_raw=pd.read_json('/home/naqib/Documents/git/data-engineering-projects/nosql-tutorial/data/data.json')
df_inspection=df_raw.copy()
df_inspection['address_id']=df_inspection.index
df_inspection=df_inspection.drop(columns=['_id', 'address'])
df_inspection.head()

,id,certificate_number,business_name,date,result,sector,address_id
0,10021-2015-ENFO,9278806,ATLIXCO DELI GROCERY INC.,2015-02-20,No Violation Issued,Cigarette Retail Dealer - 127,0
1,1012-2015-CMPL,5346909,A&C CHIMNEY CORP.,2015-04-22,Violation Issued,Home Improvement Contractor - 100,1
2,10127-2015-CMPL,5381180,ERIC CONSTRUCTION AND DECORATING INC.,2015-09-08,Violation Issued,Home Improvement Contractor - 100,2
3,10172-2015-CMPL,9304489,UNNAMED HOT DOG VENDOR LICENSE NUMBER TA01158,2015-08-21,No Violation Issued,Mobile Food Vendor - 881,3
4,102-2015-UNIT,10003479,SOUTH BRONX AUTOMOTIVE CORP,2015-05-28,Pass,Tow Truck Company - 124,4


## Normalise address columns from city_inspection

In [58]:
nested_address=df_raw['address'].copy()
df_address=pd.json_normalize(nested_address)
df_address=df_address.reset_index()
df_address['address_id']=df_address.index
# df_address=df_address.rename(columns={'index':'address_id'})
df_address=df_address.reindex(columns=['address_id','city','zip','street','number'])

# this part is used to tackle mixtypes value errors when convert to parquet file
df_address[['zip','number']]=df_address[['zip','number']].replace([''],pd.NA)
df_address[['city','street']]=df_address[['city','street']].replace([''],[None])
df_address[['address_id','street']]=df_address[['address_id','street']].astype(str)
df_address[['zip','number']]=df_address[['zip','number']].astype(str)

# df_address[df_address.isnull()==False]
df_address

,address_id,city,zip,street,number
0,0,RIDGEWOOD,11385,MENAHAN ST,1712
1,1,QUEENS VLG,11428,210TH ST,9440
2,2,STATEN ISLAND,10304,TODT HILL RD,1233
3,3,None,<NA>,None,<NA>
4,4,None,<NA>,None,<NA>
5,5,None,<NA>,None,<NA>
6,6,NEW YORK,10030,FREDRCK D BLVD,2655
7,7,NEW YORK,10030,FREDERICK DOUGLASS BLVD,2655
8,8,JERSEY CITY,7306,TONNELE AVE,70
9,9,NEW YORK,10030,FREDRCK D BLVD,2653


In [5]:
df_zips=pd.read_json('/home/naqib/Documents/git/data-engineering-projects/nosql-tutorial/data/zips.json')
df_zips

ValueError: Trailing data

## normalise zips dataframe

In [ ]:
df_loc=df_zips['loc'].copy()
lon=[]
lat=[]
for i in df_loc:
    lon.append(str(i[0]))
    lat.append(str(i[1]))
    
df_zips['lon']=lon
df_zips['lat']=lat
df_zips=df_zips.rename(columns={'_id':'zip'})
df_zips=df_zips.drop(columns='loc')
df_zips=df_zips.reindex(columns=['zip','city','state','pop','lon','lat'])
df_zips[['lon','lat']]=df_zips[['lon','lat']].astype(float)
df_zips.info()

In [13]:
import pyarrow as pa
def transform_inspection(json_path_city_inspection=None, json_path_zips=None):
    '''
    transform function for first task, aggregate between city_inspection and zips data that contains geojson
    and number of population on particular zip code
    
    return:
        list of normalised dataframe, [df_inspection ,df_address, df_zips]
    '''
    df_raw=pd.read_json('/home/naqib/Documents/git/data-engineering-projects/nosql-tutorial/data/city_inspection.json')
    df_inspection=df_raw.copy()
    df_inspection['address_id']=df_inspection.index
    df_inspection=df_inspection.drop(columns=['_id', 'address'])
    df_inspection['certificate_number']=df_inspection['certificate_number'].astype(str)
     
    nested_address=df_raw['address'].copy()
    df_address=pd.json_normalize(nested_address)
    df_address=df_address.reset_index()
    df_address['address_id']=df_address.index
    # df_address=df_address.rename(columns={'index':'address_id'})
    df_address=df_address.reindex(columns=['address_id','city','zip','street','number'])
    df_address[['zip','number']]=df_address[['zip','number']].replace([''],pd.NA)
    df_address[['city','street']]=df_address[['city','street']].replace([''],[None])
    df_address[['address_id','street']]=df_address[['address_id','street']].astype(str)
    df_address[['zip','number']]=df_address[['zip','number']].astype(str)
    
    df_zips=pd.read_json('/home/naqib/Documents/git/data-engineering-projects/nosql-tutorial/data/zips.json')
    df_loc=df_zips['loc'].copy()
    lon=[]
    lat=[]
    for i in df_loc:
        lon.append(i[0])
        lat.append(i[1])
    df_zips['lon']=lon
    df_zips['lat']=lat
    df_zips=df_zips.rename(columns={'_id':'zip'})
    df_zips=df_zips.drop(columns='loc')
    df_zips=df_zips.reindex(columns=['zip','city','state','pop','lon','lat'])
    df_zips[['lon','lat']]=df_zips[['lon','lat']].astype(float)
    df_zips['zip']=df_zips['zip'].astype(str)
    df_normalised=[df_inspection, df_address, df_zips]
    return df_normalised

# pq_address= pa.Table.from_pandas(transform_inspection()[1])
# transform_inspection()[1].astype({'street': str}).to_parquet('/tmp/address.pq',
#                                 compression='gzip')
# pa.write_table(pq_address,'/tmp/address.pq')b
transform_inspection()[1]



ValueError: Expected object or value

In [2]:
test=pd.read_parquet('/home/naqib/Documents/git/data-engineering-projects/nosql-tutorial/data/transformed_project_three_address.pq')
test=test.set_index('address_id')
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81047 entries, 0 to 81046
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   city    80899 non-null  object
 1   zip     81047 non-null  object
 2   street  81047 non-null  object
 3   number  81047 non-null  object
dtypes: object(4)
memory usage: 3.1+ MB


In [ ]:
test='P0009414'
print(str(test))

In [ ]:
tripdata=[{ "_id" : set("6032fef0b89ac067bb662a86"), "tripduration" : 316, "starttime" : "2019-01-01 00:04:43.7360", "stoptime" : "2019-01-01 00:10:00.6080", "start station id" : 519, "start station name" : "Pershing Square North", "start station latitude" : 40.751873, "start station longitude" : -73.977706, "end station id" : 518, "end station name" : "E 39 St & 2 Ave", "end station latitude" : 40.74780373, "end station longitude" : -73.9734419, "bikeid" : 32723, "usertype" : "Subscriber", "birth year" : 1964, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a87"), "tripduration" : 320, "starttime" : "2019-01-01 00:01:47.4010", "stoptime" : "2019-01-01 00:07:07.5810", "start station id" : 3160, "start station name" : "Central Park West & W 76 St", "start station latitude" : 40.77896784, "start station longitude" : -73.97374737, "end station id" : 3283, "end station name" : "W 89 St & Columbus Ave", "end station latitude" : 40.7882213, "end station longitude" : -73.97041561, "bikeid" : 15839, "usertype" : "Subscriber", "birth year" : 1971, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a88"), "tripduration" : 303, "starttime" : "2019-01-01 00:07:35.9450", "stoptime" : "2019-01-01 00:12:39.5020", "start station id" : 229, "start station name" : "Great Jones St", "start station latitude" : 40.72743423, "start station longitude" : -73.99379025, "end station id" : 503, "end station name" : "E 20 St & Park Ave", "end station latitude" : 40.73827428, "end station longitude" : -73.98751968, "bikeid" : 35379, "usertype" : "Subscriber", "birth year" : 1979, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a89"), "tripduration" : 2719, "starttime" : "2019-01-01 00:07:03.5450", "stoptime" : "2019-01-01 00:52:22.6500", "start station id" : 504, "start station name" : "1 Ave & E 16 St", "start station latitude" : 40.73221853, "start station longitude" : -73.98165557, "end station id" : 3709, "end station name" : "W 15 St & 6 Ave", "end station latitude" : 40.738046142482766, "end station longitude" : -73.99642959237099, "bikeid" : 21579, "usertype" : "Subscriber", "birth year" : 1990, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a8a"), "tripduration" : 535, "starttime" : "2019-01-01 00:08:33.1790", "stoptime" : "2019-01-01 00:17:28.3540", "start station id" : 3630, "start station name" : "Frederick Douglass Blvd & W 115 St", "start station latitude" : 40.8038654, "start station longitude" : -73.9559308, "end station id" : 3529, "end station name" : "Lenox Ave & W 130 St", "end station latitude" : 40.8107922, "end station longitude" : -73.9430681, "bikeid" : 30315, "usertype" : "Subscriber", "birth year" : 1989, "gender" : 2 },
{ "_id" : set("6032fef0b89ac067bb662a8b"), "tripduration" : 591, "starttime" : "2019-01-01 00:06:03.9970", "stoptime" : "2019-01-01 00:15:55.4380", "start station id" : 3171, "start station name" : "Amsterdam Ave & W 82 St", "start station latitude" : 40.78524672, "start station longitude" : -73.97667321, "end station id" : 3154, "end station name" : "E 77 St & 3 Ave", "end station latitude" : 40.77314236, "end station longitude" : -73.95856158, "bikeid" : 27451, "usertype" : "Subscriber", "birth year" : 1987, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a8c"), "tripduration" : 280, "starttime" : "2019-01-01 00:09:21.0060", "stoptime" : "2019-01-01 00:14:01.1510", "start station id" : 3675, "start station name" : "3 Ave & E 95 St", "start station latitude" : 40.784596798848916, "start station longitude" : -73.94968539476395, "end station id" : 3288, "end station name" : "E 88 St & 1 Ave", "end station latitude" : 40.778301, "end station longitude" : -73.9488134, "bikeid" : 35391, "usertype" : "Subscriber", "birth year" : 1987, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a8d"), "tripduration" : 1547, "starttime" : "2019-01-01 00:11:22.9070", "stoptime" : "2019-01-01 00:37:10.0550", "start station id" : 285, "start station name" : "Broadway & E 14 St", "start station latitude" : 40.73454567, "start station longitude" : -73.99074142, "end station id" : 3358, "end station name" : "Garfield Pl & 8 Ave", "end station latitude" : 40.6711978, "end station longitude" : -73.97484126, "bikeid" : 35551, "usertype" : "Subscriber", "birth year" : 1981, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a8e"), "tripduration" : 300, "starttime" : "2019-01-01 00:12:17.9310", "stoptime" : "2019-01-01 00:17:18.8300", "start station id" : 3394, "start station name" : "Columbia St & W 9 St", "start station latitude" : 40.6769993, "start station longitude" : -74.00647134, "end station id" : 3398, "end station name" : "Smith St & 9 St", "end station latitude" : 40.6746957, "end station longitude" : -73.99785768, "bikeid" : 18636, "usertype" : "Subscriber", "birth year" : 1990, "gender" : 2 },
{ "_id" : set("6032fef0b89ac067bb662a8f"), "tripduration" : 209, "starttime" : "2019-01-01 00:15:09.9730", "stoptime" : "2019-01-01 00:18:39.0110", "start station id" : 438, "start station name" : "St Marks Pl & 1 Ave", "start station latitude" : 40.72779126, "start station longitude" : -73.98564945, "end station id" : 301, "end station name" : "E 2 St & Avenue B", "end station latitude" : 40.72217444, "end station longitude" : -73.98368779, "bikeid" : 34416, "usertype" : "Subscriber", "birth year" : 1998, "gender" : 2 },
{ "_id" : set("6032fef0b89ac067bb662a90"), "tripduration" : 123, "starttime" : "2019-01-01 00:12:19.6030", "stoptime" : "2019-01-01 00:14:23.1040", "start station id" : 432, "start station name" : "E 7 St & Avenue A", "start station latitude" : 40.72621788, "start station longitude" : -73.98379855, "end station id" : 3656, "end station name" : "E 2 St & Avenue A", "end station latitude" : 40.72307749068673, "end station longitude" : -73.98583620786667, "bikeid" : 35206, "usertype" : "Subscriber", "birth year" : 1987, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a91"), "tripduration" : 932, "starttime" : "2019-01-01 00:14:02.7970", "stoptime" : "2019-01-01 00:29:35.0450", "start station id" : 379, "start station name" : "W 31 St & 7 Ave", "start station latitude" : 40.749156, "start station longitude" : -73.9916, "end station id" : 336, "end station name" : "Sullivan St & Washington Sq", "end station latitude" : 40.73047747, "end station longitude" : -73.99906065, "bikeid" : 35148, "usertype" : "Subscriber", "birth year" : 1986, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a92"), "tripduration" : 976, "starttime" : "2019-01-01 00:15:12.8890", "stoptime" : "2019-01-01 00:31:29.1450", "start station id" : 3452, "start station name" : "Bayard St & Leonard St", "start station latitude" : 40.71915571696044, "start station longitude" : -73.94885390996933, "end station id" : 251, "end station name" : "Mott St & Prince St", "end station latitude" : 40.72317958, "end station longitude" : -73.99480012, "bikeid" : 35685, "usertype" : "Subscriber", "birth year" : 1994, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a93"), "tripduration" : 97, "starttime" : "2019-01-01 00:15:40.0140", "stoptime" : "2019-01-01 00:17:17.3120", "start station id" : 3430, "start station name" : "Richardson St & N Henry St", "start station latitude" : 40.71907891179564, "start station longitude" : -73.94223690032959, "end station id" : 3095, "end station name" : "Graham Ave & Herbert St", "end station latitude" : 40.71929301, "end station longitude" : -73.94500379, "bikeid" : 34307, "usertype" : "Subscriber", "birth year" : 1988, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a94"), "tripduration" : 467, "starttime" : "2019-01-01 00:16:22.9480", "stoptime" : "2019-01-01 00:24:10.5430", "start station id" : 507, "start station name" : "E 25 St & 2 Ave", "start station latitude" : 40.73912601, "start station longitude" : -73.97973776, "end station id" : 492, "end station name" : "W 33 St & 7 Ave", "end station latitude" : 40.75019995, "end station longitude" : -73.99093085, "bikeid" : 35561, "usertype" : "Subscriber", "birth year" : 1989, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a95"), "tripduration" : 348, "starttime" : "2019-01-01 00:17:28.9820", "stoptime" : "2019-01-01 00:23:17.6080", "start station id" : 3095, "start station name" : "Graham Ave & Herbert St", "start station latitude" : 40.71929301, "start station longitude" : -73.94500379, "end station id" : 3101, "end station name" : "N 12 St & Bedford Ave", "end station latitude" : 40.72079821, "end station longitude" : -73.95484712, "bikeid" : 35695, "usertype" : "Subscriber", "birth year" : 1988, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a96"), "tripduration" : 505, "starttime" : "2019-01-01 00:18:45.1180", "stoptime" : "2019-01-01 00:27:10.5990", "start station id" : 3132, "start station name" : "E 59 St & Madison Ave", "start station latitude" : 40.76350532, "start station longitude" : -73.97109243, "end station id" : 359, "end station name" : "E 47 St & Park Ave", "end station latitude" : 40.75510267, "end station longitude" : -73.97498696, "bikeid" : 31801, "usertype" : "Subscriber", "birth year" : 1981, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a97"), "tripduration" : 3494, "starttime" : "2019-01-01 00:18:57.5640", "stoptime" : "2019-01-01 01:17:11.5700", "start station id" : 3171, "start station name" : "Amsterdam Ave & W 82 St", "start station latitude" : 40.78524672, "start station longitude" : -73.97667321, "end station id" : 3164, "end station name" : "Columbus Ave & W 72 St", "end station latitude" : 40.7770575, "end station longitude" : -73.97898475, "bikeid" : 35785, "usertype" : "Subscriber", "birth year" : 1954, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a98"), "tripduration" : 451, "starttime" : "2019-01-01 00:21:00.6320", "stoptime" : "2019-01-01 00:28:31.9450", "start station id" : 403, "start station name" : "E 2 St & 2 Ave", "start station latitude" : 40.72502876, "start station longitude" : -73.99069656, "end station id" : 545, "end station name" : "E 23 St & 1 Ave", "end station latitude" : 40.736502, "end station longitude" : -73.97809472, "bikeid" : 32038, "usertype" : "Subscriber", "birth year" : 1985, "gender" : 1 },
{ "_id" : set("6032fef0b89ac067bb662a99"), "tripduration" : 829, "starttime" : "2019-01-01 00:19:08.1030", "stoptime" : "2019-01-01 00:32:57.3880", "start station id" : 3165, "start station name" : "Central Park West & W 72 St", "start station latitude" : 40.77579376683666, "start station longitude" : -73.9762057363987, "end station id" : 3295, "end station name" : "Central Park W & W 96 St", "end station latitude" : 40.79127, "end station longitude" : -73.964839, "bikeid" : 32106, "usertype" : "Subscriber", "birth year" : 1969, "gender" : 0 }]


## Format columns name

In [ ]:
# def transform_tripdata(tripdata_file_path=None):
#     '''
#     transform function task 2, format schema for bigquery

#     return:
#         tripdata dataframe
#     '''
df_raw=pd.DataFrame(tripdata)
cols=df_raw.columns
def format_columns(df):
    cols=df.columns
    cols=cols.str.lower()
    cols=cols.str.replace(' ','_', regex=True)
    
    return cols
df_tripdata=df_raw.copy()
df_tripdata.columns=format_columns(df_tripdata)
df_raw.info()

## Change dataypes accordingly 

- columns that have date/time values to datetime datatypes
- columns that have unique id to object 

In [ ]:
df_tripdata['starttime']=pd.to_datetime(df_raw['starttime'])
df_tripdata['stoptime']=pd.to_datetime(df_raw['stoptime'])
df_tripdata['start_station_id']=df_tripdata['start_station_id'].astype(str)
df_tripdata['end_station_id']=df_tripdata['end_station_id'].astype(str)
df_tripdata['bikeid']=df_tripdata['bikeid'].astype(str)
df_tripdata['end_station_id']=df_tripdata['end_station_id'].astype(str)

# df_tripdata

df_tripdata.info()

In [ ]:
df_tripdata.head(2)

In [ ]:
def transform_tripdata(tripdata_file_path=None):
    '''
    transform function task 2, format schema for bigquery

    return:
        tripdata dataframe
    '''
    df_raw=pd.DataFrame(tripdata)
    cols=df_raw.columns
    def format_columns(df):
        cols=df.columns
        cols=cols.str.lower()
        cols=cols.str.replace(' ','_', regex=True)

        return cols
    df_tripdata=df_raw.copy()
    df_tripdata.columns=format_columns(df_tripdata)
    
    df_tripdata['starttime']=pd.to_datetime(df_raw['starttime'])
    df_tripdata['stoptime']=pd.to_datetime(df_raw['stoptime'])
    df_tripdata['start_station_id']=df_tripdata['start_station_id'].astype(str)
    df_tripdata['end_station_id']=df_tripdata['end_station_id'].astype(str)
    df_tripdata['bikeid']=df_tripdata['bikeid'].astype(str)
    df_tripdata['end_station_id']=df_tripdata['end_station_id'].astype(str)

    return df_tripdata

In [ ]:
test=pd.read_parquet('')